# Laboratorio de regresión - 4

|                |   |
:----------------|---|
| **Nombre**     |  Julieta Madrigal Flores |
| **Fecha**      |  Lunes 9 de febrero del 2026 |
| **Expediente** |  744029 |

## Modelos penalizados

Hasta ahora la función de costo que usamos para decidir qué tan bueno es nuestro modelo al momento de ajustar es:

$$ \text{RSS} = \sum_{i=1}^n e_i^2 = \sum_{i=1}^n (y_i - \hat{y_i})^2 $$

Dado que los errores obtenidos son una combinación de sesgo y varianza, puede ser que se sesgue un parámetro para minimizar el error. Esto significa que el modelo puede decidir que la salida no sea una combinación de los factores, sino una fuerte predilección sobre uno de los factores solamente.

E.g. se quiere ajustar un modelo

$$ \hat{z} = \hat{\beta_0} + \hat{\beta_1} x + \hat{\beta_2} y $$

Se ajusta el modelo y se decide que la mejor decisión es $\hat{\beta_1} = 10000$ y $\hat{\beta_2}=50$. Considera limitaciones de problemas reales:
- Quizás los parámetros son ajustes de maquinaria que se deben realizar para conseguir el mejor producto posible, y que $10000$ sea imposible de asignar.
- Quizás los datos actuales están sesgados y sólo hacen parecer que uno de los factores importa más que el otro.

Una de las formas en las que se puede mitigar este problema es penalizando a los parámetros del modelo, cambiando la función de costo:

$$ \text{RSS}_{L2} = \sum_{i=1}^n e_i^2  + \lambda \sum_{j=1}^p \hat{\beta_j}^2 $$

El *L2* significa que se está agregando una penalización de segundo orden. Lo que hace esta penalización es que los factores ahora sólo tendrán permitido crecer si hay una reducción al menos proporcional en el error (sacrificamos sesgo, pero reducimos la varianza).

Asimismo, existe la penalización *L1*

$$ \text{RSS}_{L1} = \sum_{i=1}^n e_i^2  + \lambda \sum_{j=1}^p |\hat{\beta_j}| $$

A las penalizaciones *L2* y *L1* se les conoce también como Ridge y Lasso, respectivamente.

Para realizar una regresión con penalización de Ridge o de Lasso usamos el objeto `Ridge(alpha=?)` o `Lasso(alpha=?)` en lugar de `LinearRegression()` de `sklearn`.

Utiliza el dataset de publicidad (Advertising.csv) y realiza 3 regresiones múltiples:

$$ \text{sales} = \beta_0 + \beta_1 (\text{TV}) + \beta_2 (\text{radio}) + \beta_3 (\text{newspaper}) + \epsilon $$

1. Sin penalización
2. Con penalización L2
3. Con penalización L1

¿Qué puedes observar al ajustar los valores de `alpha`?

Compara los resultados de los coeficientes utilizando valores diferentes de $\alpha$ y los $R^2$ resultantes.



In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('Advertising.csv')
data.head()

,Unnamed: 0,TV,radio,newspaper,sales
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9


In [3]:
y = data['sales']
y = np.array(y).reshape(-1, 1)

In [4]:
x = data['TV']
x = np.array(x).reshape(-1, 1)

In [5]:
r = data['radio']
r = np.array(r).reshape(-1, 1)

In [6]:
news = data['newspaper']
news = np.array(news).reshape(-1, 1)

In [7]:
# Hacemos la matriz con TV, radio y newspaper
X = np.hstack((x, r, news))
# agregamos la columna de unos
Xunos = np.column_stack([np.ones(len(X)), X])

***Sin penalización***

In [8]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(Xunos, y)

LinearRegression()

In [9]:
lr.coef_

array([[ 0.        ,  0.04576465,  0.18853002, -0.00103749]])

In [10]:
lr.intercept_

array([2.93888937])

In [11]:
import statsmodels.api as sm
model_multi = sm.OLS(y, Xunos )
results_multi = model_multi.fit()
results_multi.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.897
Model:                            OLS   Adj. R-squared:                  0.896
Method:                 Least Squares   F-statistic:                     570.3
Date:                Fri, 06 Feb 2026   Prob (F-statistic):           1.58e-96
Time:                        22:39:43   Log-Likelihood:                -386.18
No. Observations:                 200   AIC:                             780.4
Df Residuals:                     196   BIC:                             793.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.9389      0.312      9.422      0.000       2.324       3.554
x1             0.0458      0.001     32.809      0.000       0.043       0.049
x2             0.1885      0.009     21.893      0.000       0.172       0.206
x3            -0.0010      0.006     -0.177      0.860      -0.013       0.011
==============================================================================
Omnibus:                       60.414   Durbin-Watson:                   2.084
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              151.241
Skew:                          -1.327   Prob(JB):                     1.44e-33
Kurtosis:                       6.332   Cond. No.                         454.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [45]:
r2 = results_multi.rsquared
r2

np.float64(0.8972106381789522)

In [13]:
# Aquí NOO se usa la matriz de unos
# Train = donde el modelo aprende y sirve para estimar los betas
# Test = datos "nuevos" que el modelo NO considera pero con los que va a comparar sus estimaciones de TRAIN

# Test = 30 por lo que train = 70, normalmente se usa 80/20 pero como el dataset es pequeño, se usa 70/30
# random_state = fijamos una semilla para siempre empezar donde mismo, sin ella cada que se corra va a dar un resultado diferente
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


##***Con penalización L2 - Ridge***

In [12]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

- α es un número que controla qué tan fuerte es el castigo en Ridge


**Para α = 0.01**


In [31]:
modelo_ridge_01 = Ridge(alpha=0.01)
modelo_ridge_01.fit(X_train, y_train)
r2_01 = modelo_ridge_01.score(X_test, y_test)

modelo_ridge_01.intercept_, modelo_ridge_01.coef_, r2_01

(array([2.7089503]),
 array([0.04405928, 0.19928742, 0.00688247]),
 0.8609466894165918)

**Para α = 0.1**

In [32]:
modelo_ridge_0_1 = Ridge(alpha=0.1)
modelo_ridge_0_1.fit(X_train, y_train)
r2_0_1 = modelo_ridge_0_1.score(X_test, y_test)

modelo_ridge_0_1.intercept_, modelo_ridge_0_1.coef_, r2_0_1

(array([2.70896119]),
 array([0.04405928, 0.19928674, 0.00688262]),
 0.8609470367537362)

**Para α = 1**

In [21]:
# α ≥ 0 es el único requisito
# α pequeño → castigo pequeño, es decir el modelo casi no cambia y se parece mucho a regresión normal OLs

modelo_ridge_1 = Ridge(alpha=1.0)
modelo_ridge_1.fit(X_train, y_train)

Ridge()

In [22]:
modelo_ridge_1.intercept_

array([2.70907007])

In [23]:
modelo_ridge_1.coef_

array([0.04405931, 0.19927997, 0.00688415])

In [26]:
r2_1 = modelo_ridge_1.score(X_test, y_test)
r2_1


0.8609505096450261

**Para α = 10**

In [33]:
modelo_ridge_10 = Ridge(alpha=10)
modelo_ridge_10.fit(X_train, y_train)
r2_10 = modelo_ridge_10.score(X_test, y_test)

modelo_ridge_10.intercept_, modelo_ridge_10.coef_, r2_10

(array([2.71015854]),
 array([0.04405956, 0.19921229, 0.00689942]),
 0.8609851905762931)

##***Con penalización L1 - Lasso***

In [34]:
from sklearn.linear_model import Lasso

**Para α = 0.01**

In [38]:
modelo_lasso_01 = Lasso(alpha=0.01)
modelo_lasso_01.fit(X_train, y_train)
r2_lasso_01 = modelo_lasso_01.score(X_test, y_test)

modelo_lasso_01.intercept_, modelo_lasso_01.coef_, r2_lasso_01


(array([2.71044116]),
 array([0.04405837, 0.19924811, 0.00686858]),
 0.860990563063074)

**Para α = 0.1**

In [39]:
modelo_lasso_0_1 = Lasso(alpha=0.1)
modelo_lasso_0_1.fit(X_train, y_train)
r2_lasso_0_1 = modelo_lasso_0_1.score(X_test, y_test)

modelo_lasso_0_1.intercept_, modelo_lasso_0_1.coef_, r2_lasso_0_1

(array([2.72389096]),
 array([0.04405065, 0.19888746, 0.00674515]),
 0.861388098941966)

**Para α = 1**

In [41]:
modelo_lasso_1 = Lasso(alpha=1)
modelo_lasso_1.fit(X_train, y_train)
r2_lasso_1 = modelo_lasso_1.score(X_test, y_test)

modelo_lasso_1.intercept_, modelo_lasso_1.coef_, r2_lasso_1

(array([2.85838896]),
 array([0.04397345, 0.19528096, 0.00551083]),
 0.8651496069722558)

**Para α = 10**

In [42]:
modelo_lasso_10 = Lasso(alpha=10)
modelo_lasso_10.fit(X_train, y_train)
r2_lasso_10 = modelo_lasso_10.score(X_test, y_test)

modelo_lasso_10.intercept_, modelo_lasso_10.coef_, r2_lasso_10

(array([4.07562878]),
 array([0.04318352, 0.15592236, 0.        ]),
 0.8791415613357129)

##***Tabla de comparación***

In [48]:
table = pd.DataFrame(index = ['α', 'Ridge', 'Lasso'],
                        data = [['0.01', '0.1', '1', '10'],
                         [r2_01, r2_0_1, r2_1, r2_10], [r2_lasso_01, r2_lasso_0_1, r2_lasso_1, r2_lasso_10]])
table.T

,α,Ridge,Lasso
0,0.01,0.860947,0.860991
1,0.1,0.860947,0.861388
2,1,0.860951,0.86515
3,10,0.860985,0.879142


In [50]:
print(f"El valor R² de la regresión sin penalización fue de: {r2}")

El valor R² de la regresión sin penalización fue de: 0.8972106381789522


**CONCLUSIÓN**

Los resultados demuestran que Ridge mantiene una R² bastante constante en los distintos valores de α, esto significa que penalizar L2 no afecta de manera significativa el desempeño del modelo. Mientras que Lasso incrementa su R² conforme va aumentando α, esto sugiere que penalizar L1 permite que el modelo se vaya haciendo más simple y sencillo al ir eliminando variables poco relevantes. Finalmente es importante mencionar que después de probar con distintos valores de α en Ridge y Lasso, la regresión sin penalización obtuvo una mejor R², por lo que se podría concluir que el mejor modelo fue la regresión sin penalización. Sin embargo, si se busca mejorar aún más la R², se podría probar con valores de α más altos con el método Lasso, el cual mostró mayor sensibilidad al aumento de α.

***NOTA***

**RIDGE** = Castiga los coeficientes beta grandes pero no los hace cero, solo los reducen un poco.

**LASSO** = Castiga de forma más agresiva, hace que algunos coeficientes se vuelven exactamente cero y a partir de eso, hacer selección de variables.

La penalización en Ridge y Lasso se aplica sobre los coeficientes beta  del modelo, con el objetivo de reducir la complejidad del modelo, no le ponga “pesos enormes” a una sola variable = el modelo no exagere cuánto influye una variable y por ende, sea más estable.

Si los coeficientes son **MUY GRANDES** :el modelo se vuelve muy sensible porque pequeños cambios en *x* producen cambios grandes en *y* y el modelo sufre de sobreajuste.

